In [38]:
# imports

import torch 
import torch.nn as nn
import torch.optim as optimizer
import torchvision 
import torchvision.transforms as trf


In [49]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1)
        self.relu = nn.ReLU()
        self.maxPool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding = 1)
        self.fc = nn.Linear(32 * 8 * 8, 10)

    def forward(self,image):

        # 1st Convolution Layer
        image = self.conv1(image)
        image = self.relu(image)
        image = self.maxPool(image)

        # 2nd Convolution Layer
        image = self.conv2(image)
        image = self.relu(image)
        image = self.maxPool(image)

        # Flattening the image/tensor object to one dimension
        image = image.view(image.size(0),-1)
        image = self.fc(image)

        return image

In [40]:
def train(model, trainLoader, criterion, optimizer, device):
    
    model.train()

    for images, labels in trainLoader:

        # sending the images and lables to the specified device
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs,labels)

        loss.backward()

        optimizer.step()


In [41]:
def evaluate(model, test_loader, device):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images,labels = images.to(device), labels.to(device)
            outputs = model(images)

            _,predicted = torch.max(outputs.data,1)

            total += labels.size(0)
            labels += (predicted == labels).sum().item()
    acc = 100 * correct /  total

    return acc

In [42]:
transform = trf.Compose([
    trf.ToTensor(),
    trf.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [43]:
tr_set = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
train_loader = torch.utils.data.DataLoader(tr_set,batch_size=64,shuffle=True,num_workers=2)

Files already downloaded and verified


In [44]:
ts_set = torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
test_loader = torch.utils.data.DataLoader(ts_set,batch_size=64,shuffle=True,num_workers=2)

Files already downloaded and verified


In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [46]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optim = optimizer.SGD(model.parameters(),lr=0.01,momentum=0.9) 

In [50]:
for epoch in range(2):
    train(model, trainLoader=train_loader,criterion=criterion,optimizer=optim, device=device)
    acc = evaluate(model, test_loader,device=device)
    print(f"Epoch: {epoch + 1}. Accuracy: {acc:.2f}%")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x1568 and 2048x10)